In [1]:
"https://api.scryfall.com/cards/search?order=set&unique=art&q=set%3A'eld'+lang%3A'en'"

"https://api.scryfall.com/cards/search?order=set&unique=art&q=set%3A'eld'+lang%3A'en'"

In [2]:
# Change dir to get SQL connector
import os
os.chdir("..")
from src.sqldb import SQLDatabase

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import re
import seaborn as sns

# Updating Tables

In [4]:
FLAT_FILE_DIR = "flat_files/"

full_table = pd.read_csv(FLAT_FILE_DIR + "full_table.csv")
event_table = pd.read_csv(FLAT_FILE_DIR + "event_table.csv")
deck_list_table = pd.read_csv(FLAT_FILE_DIR + "deck_list_table.csv")
pilot_table = pd.read_csv(FLAT_FILE_DIR + "pilot_table.csv")
deck_table = pd.read_csv(FLAT_FILE_DIR + "deck_table.csv")
card_table = pd.read_csv(FLAT_FILE_DIR + "card_table.csv")

updater = m.Updater()
updater.update_cards()

# Cleaning

New decks need to be checked for misformed codes & missing archetypes/categories

In [5]:
deck_table.loc[deck_table.archetype.isna()]

,deckId,eventId,pilotId,deckUrl,name,rank,archetype,category


In [6]:
deck_table.loc[deck_table.category.isna()]

,deckId,eventId,pilotId,deckUrl,name,rank,archetype,category


In [7]:
unmatched = set(deck_list_table.loc[~(deck_list_table.cardId.isin(card_table.cardId))]["cardId"])

def find_urls_with_unmatched(unmatched):
    unmatched_deck_ids = list(deck_list_table.loc[deck_list_table.cardId == unmatched]["deckId"])
    return set(deck_table.loc[deck_table.deckId.isin(unmatched_deck_ids)]["deckUrl"])

def print_page_source(unmatched):
    unmatched_url = find_urls_with_unmatched(unmatched).pop()
    print(unmatched_url)
    html = requests.get("https://www.mtgtop8.com/event" + unmatched_url)
    return BeautifulSoup(html.text, features="lxml")

In [8]:
unmatched

{nan}

In [9]:
standard_sets = "eld|thb|znr|iko|khm|m21"
full_table[
    (~full_table["cardId"].str.contains(standard_sets, regex=True))
    & (full_table["date"] >= "2020-09-29")
][["cardId", "name"]].drop_duplicates()

,cardId,name
9598,077grn,Midnight Reaper
14906,101m13,Murder


In [17]:
deck_table[
    deck_table["deckId"].isin(
        full_table[
            full_table["cardId"] == "101m13"
        ]
        ["deckId"]
    )
]

,deckId,eventId,pilotId,deckUrl,name,rank,archetype,category
552,553,102,479,?e=27633&d=419513&f=ST,Jund,2,jund,jund
